In [58]:
!pip install tensorflow-text==2.13.0
!pip install transformers
!pip install torch
!pip install nltk

In [34]:
import pandas as pd
import numpy as np

import re
import tensorflow as tf
print(tf.__version__)
import tensorflow_hub as hub
import tensorflow_text as text


from transformers import TFBertModel, BertTokenizer

2.13.0


In [29]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
data_movie_details=pd.read_json('/content/drive/MyDrive/Project/Spoiler-Blocker-Extension/Data/IMDB_movie_details.json',lines=True)
data_movie_details.head(15)

,movie_id,plot_summary,duration,genre,rating,release_date,plot_synopsis
0,tt0105112,"Former CIA analyst, Jack Ryan is in England wi...",1h 57min,"[Action, Thriller]",6.9,1992-06-05,"Jack Ryan (Ford) is on a ""working vacation"" in..."
1,tt1204975,"Billy (Michael Douglas), Paddy (Robert De Niro...",1h 45min,[Comedy],6.6,2013-11-01,Four boys around the age of 10 are friends in ...
2,tt0243655,"The setting is Camp Firewood, the year 1981. I...",1h 37min,"[Comedy, Romance]",6.7,2002-04-11,
3,tt0040897,"Fred C. Dobbs and Bob Curtin, both down on the...",2h 6min,"[Adventure, Drama, Western]",8.3,1948-01-24,Fred Dobbs (Humphrey Bogart) and Bob Curtin (T...
4,tt0126886,Tracy Flick is running unopposed for this year...,1h 43min,"[Comedy, Drama, Romance]",7.3,1999-05-07,Jim McAllister (Matthew Broderick) is a much-a...
5,tt0286716,"Bruce Banner, a brilliant scientist with a clo...",2h 18min,"[Action, Sci-Fi]",5.7,2003-06-20,Bruce Banner (Eric Bana) is a research scienti...
6,tt0090605,57 years after Ellen Ripley had a close encoun...,2h 17min,"[Action, Adventure, Sci-Fi]",8.4,1986-07-18,"After the opening credits, we see a spacecraft..."
7,tt0243155,Bridget Jones is an average woman struggling a...,1h 37min,"[Comedy, Drama, Romance]",6.7,2001-04-13,Bridget Jones (adorably played by Renee Zellwe...
8,tt0121765,"Ten years after the invasion of Naboo, the Gal...",2h 22min,"[Action, Adventure, Fantasy]",6.6,2002-05-16,The opening crawl reveals that the Galactic Re...
9,tt0443453,Borat Sagdiyev is a TV reporter of a popular s...,1h 24min,[Comedy],7.3,2006-11-03,Borat Sagdiyev is a TV reporter of a popular s...


In [31]:
data_movie_reviews=pd.read_json('/content/drive/MyDrive/Project/Spoiler-Blocker-Extension/Data/IMDB_reviews.json',lines=True)
data_movie_reviews

,review_date,movie_id,user_id,is_spoiler,review_text,rating,review_summary
0,10 February 2006,tt0111161,ur1898687,True,"In its Oscar year, Shawshank Redemption (writt...",10,A classic piece of unforgettable film-making.
1,6 September 2000,tt0111161,ur0842118,True,The Shawshank Redemption is without a doubt on...,10,Simply amazing. The best film of the 90's.
2,3 August 2001,tt0111161,ur1285640,True,I believe that this film is the best story eve...,8,The best story ever told on film
3,1 September 2002,tt0111161,ur1003471,True,"**Yes, there are SPOILERS here**This film has ...",10,Busy dying or busy living?
4,20 May 2004,tt0111161,ur0226855,True,At the heart of this extraordinary movie is a ...,8,"Great story, wondrously told and acted"
...,...,...,...,...,...,...,...
573908,8 August 1999,tt0139239,ur0100166,False,"Go is wise, fast and pure entertainment. Assem...",10,The best teen movie of the nineties
573909,31 July 1999,tt0139239,ur0021767,False,"Well, what shall I say. this one´s fun at any ...",9,Go - see the movie
573910,20 July 1999,tt0139239,ur0392750,False,"Go is the best movie I have ever seen, and I'v...",10,It's the best movie I've ever seen
573911,11 June 1999,tt0139239,ur0349105,False,Call this 1999 teenage version of Pulp Fiction...,3,Haven't we seen this before?


In [32]:
data_movie_reviews.iloc[0][4]

'In its Oscar year, Shawshank Redemption (written and directed by Frank Darabont, after the novella Rita Hayworth and the Shawshank Redemption, by Stephen King) was nominated for seven Academy Awards, and walked away with zero. Best Picture went to Forrest Gump, while Shawshank and Pulp Fiction were "just happy to be nominated." Of course hindsight is 20/20, but while history looks back on Gump as a good film, Pulp and Redemption are remembered as some of the all-time best. Pulp, however, was a success from the word "go," making a huge splash at Cannes and making its writer-director an American master after only two films. For Andy Dufresne and Co., success didn\'t come easy. Fortunately, failure wasn\'t a life sentence.After opening on 33 screens with take of $727,327, the $25M film fell fast from theatres and finished with a mere $28.3M. The reasons for failure are many. Firstly, the title is a clunker. While iconic to fans today, in 1994, people knew not and cared not what a \'Shaws

In [35]:
df_reviews = data_movie_reviews.merge(data_movie_details, on="movie_id", how="left", suffixes=('_review','_movie'))

In [36]:
summaries = pd.Series([re.split('\s*Written by\s*\n', str(plot))[0] for plot in df_reviews['plot_summary']])
synopses = pd.Series([str(synopsis) for synopsis in df_reviews['plot_synopsis']])

In [37]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 573913 entries, 0 to 573912
Data columns (total 13 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   review_date     573913 non-null  object 
 1   movie_id        573913 non-null  object 
 2   user_id         573913 non-null  object 
 3   is_spoiler      573913 non-null  bool   
 4   review_text     573913 non-null  object 
 5   rating_review   573913 non-null  int64  
 6   review_summary  573913 non-null  object 
 7   plot_summary    573906 non-null  object 
 8   duration        573906 non-null  object 
 9   genre           573906 non-null  object 
 10  rating_movie    573906 non-null  float64
 11  release_date    573906 non-null  object 
 12  plot_synopsis   573906 non-null  object 
dtypes: bool(1), float64(1), int64(1), object(10)
memory usage: 57.5+ MB


In [38]:
df_train=pd.DataFrame(columns=['movie_id','sentence_1','sentence_2','label'])

In [39]:
df_train['movie_id'] = df_reviews['movie_id']

In [40]:
df_train['sentence_1'] = pd.Series([synopses[i] if len(synopses[i])>len(summaries[i]) else summaries[i] \
                                  for i in range(len(synopses))])
df_train['sentence_2'] = df_reviews['review_summary'] + ' ' + df_reviews['review_text']
df_train['label'] = df_reviews['is_spoiler']

In [41]:
df_train

,movie_id,sentence_1,sentence_2,label
0,tt0111161,"In 1947, Andy Dufresne (Tim Robbins), a banker...",A classic piece of unforgettable film-making. ...,True
1,tt0111161,"In 1947, Andy Dufresne (Tim Robbins), a banker...",Simply amazing. The best film of the 90's. The...,True
2,tt0111161,"In 1947, Andy Dufresne (Tim Robbins), a banker...",The best story ever told on film I believe tha...,True
3,tt0111161,"In 1947, Andy Dufresne (Tim Robbins), a banker...","Busy dying or busy living? **Yes, there are SP...",True
4,tt0111161,"In 1947, Andy Dufresne (Tim Robbins), a banker...","Great story, wondrously told and acted At the ...",True
...,...,...,...,...
573908,tt0139239,"The film is told out of chronological format, ...",The best teen movie of the nineties Go is wise...,False
573909,tt0139239,"The film is told out of chronological format, ...","Go - see the movie Well, what shall I say. thi...",False
573910,tt0139239,"The film is told out of chronological format, ...",It's the best movie I've ever seen Go is the b...,False
573911,tt0139239,"The film is told out of chronological format, ...",Haven't we seen this before? Call this 1999 te...,False


In [42]:
df_train_1=df_train.drop(['movie_id','sentence_1'],axis =1)
print(type(df_train_1))

<class 'pandas.core.frame.DataFrame'>


In [94]:
df_train_1['label'].value_counts()

False    422989
True     150924
Name: label, dtype: int64

In [95]:
df_True=df_train_1[df_train_1['label']==True]
df_True

,sentence_2,label
0,A classic piece of unforgettable film-making. ...,True
1,Simply amazing. The best film of the 90's. The...,True
2,The best story ever told on film I believe tha...,True
3,"Busy dying or busy living? **Yes, there are SP...",True
4,"Great story, wondrously told and acted At the ...",True
...,...,...
573640,Spoiler Warning! It was a mistake to have rent...,True
573641,"""Pulp Fiction""-lite... The film without a doub...",True
573642,making my generation look dumber This movie is...,True
573643,"Deceiving cover, not a bad movie but... I got ...",True


In [96]:
df_False=df_train_1[df_train_1['label']==False]

In [97]:
df_True=df_True.sample(n=10000,random_state=41)
df_False=df_False.sample(n=10000,random_state=40)

In [100]:
downsampled_df = pd.concat([df_True,df_False])
downsampled_df = downsampled_df.sample(frac=1,random_state=101).reset_index(drop=True)
downsampled_df

,sentence_2,label
0,"Good Movie!! Andy Dufresne, a successful banke...",True
1,Bram Stoker's Count Dracula Dracula has never ...,True
2,Excellent One of the fist proper thriller film...,False
3,"A Refreshingly, Lightly Cynical, But Uneven Mo...",False
4,We are Rats in a Maze Without an Exit The open...,False
...,...,...
19995,Not impressed Don't understand the high rating...,True
19996,Excellence in Truth This weekend I had the gri...,True
19997,"Hopes, Dreams & Nightmares. Almost Famous is C...",False
19998,This Movie is the Ultimate Thriller of the Mil...,False


In [106]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords


def clean_text(text):
    # Remove HTML tags (if any)
    text = re.sub(r'<.*?>', '', text)

    # Remove special characters, punctuation, and convert to lowercase
    text = re.sub(r'[^a-zA-Z\s]', '', text).lower()

    # Tokenize the text (split into words)
    tokens = text.split()

    # # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # Join the tokens back into a cleaned text
    cleaned_text = ' '.join(tokens)

    return cleaned_text
downsampled_df['sentence_2']=downsampled_df['sentence_2'].apply(clean_text)
downsampled_df

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,sentence_2,label
0,good movie andy dufresne successful banker con...,True
1,bram stokers count dracula dracula never looke...,True
2,excellent one fist proper thriller films ever ...,False
3,refreshingly lightly cynical uneven movie down...,False
4,rats maze without exit opening voiceover narra...,False
...,...,...
19995,impressed dont understand high ratings movie f...,True
19996,excellence truth weekend grim opportunity see ...,True
19997,hopes dreams nightmares almost famous cameron ...,False
19998,movie ultimate thriller millenium best thrille...,False


In [68]:
from sklearn.model_selection import train_test_split

In [69]:
PRE_TRAINED_MODEL_NAME = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME,do_lower_case = True)

def encoder(sentences):
  ids = []
  for sentence in sentences:
    encoding = tokenizer.encode_plus(
    sentence,
    max_length=16,
    truncation = True,
    add_special_tokens=True,
    return_token_type_ids=False,
    pad_to_max_length=True,
    return_attention_mask=False)
    ids.append(encoding['input_ids'])
  return ids

#Train test split
sentences=downsampled_df.sentence_2.values
labels=downsampled_df.label.values
train_sents,test_sents, train_labels, test_labels  = train_test_split(sentences,labels,test_size=0.20)

train_ids = encoder(train_sents)
print('Train Done')
test_ids = encoder(test_sents)
print('Test Done')

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Train Done
Test Done


In [107]:
X_train,X_test,y_train,y_test=train_test_split(downsampled_df['sentence_2'],downsampled_df['label'],test_size=0.2)

In [70]:
train_ids = tf.convert_to_tensor(train_ids)
test_ids = tf.convert_to_tensor(test_ids)
test_labels = tf.convert_to_tensor(test_labels)
train_labels = tf.convert_to_tensor(train_labels)

In [88]:
bert_preprocess= hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
print('Encoder being downloaded')
bert_encoder=hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

Encoder being downloaded


In [109]:
# bert_encoder = TFBertModel.from_pretrained('bert-base-uncased')
# input_word_ids = tf.keras.Input(shape=(16,), dtype=tf.int32, name="input_word_ids")
# embedding = bert_encoder([input_word_ids])
# dense = tf.keras.layers.Lambda(lambda seq: seq[:, 0, :])(embedding[0])
# dense = tf.keras.layers.Dense(128, activation='relu')(dense)
# dense = tf.keras.layers.Dropout(0.2)(dense)
# output = tf.keras.layers.Dense(1, activation='sigmoid')(dense)

# model = tf.keras.Model(inputs=[input_word_ids], outputs=output)




#BERT Layers
text_input=tf.keras.layers.Input(shape=(),dtype=tf.string,name="text")
preprocessed_text=bert_preprocess(text_input)
outputs=bert_encoder(preprocessed_text)

#Neural Network Layers
l=tf.keras.layers.Dropout(0.1,name='dropout')(outputs['pooled_output'])
l=tf.keras.layers.Dense(1,activation='sigmoid',name='output')(l)

#Construct Final Model
model=tf.keras.Model(inputs=[text_input],outputs=[l])

In [110]:
model.summary()

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 text (InputLayer)           [(None,)]                    0         []                            
                                                                                                  
 keras_layer_2 (KerasLayer)  {'input_type_ids': (None,    0         ['text[0][0]']                
                             128),                                                                
                              'input_mask': (None, 128)                                           
                             , 'input_word_ids': (None,                                           
                              128)}                                                               
                                                                                            

In [111]:
METRICS=[
    tf.keras.metrics.BinaryAccuracy(name='accuracy'),
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall')
]
model.compile(optimizer='adam',
             loss='binary_crossentropy',
             metrics=['accuracy'])

In [ ]:
history = model.fit(x = X_train, y = y_train, epochs = 10, verbose = 1, batch_size = 32, validation_data = (X_test,y_test))

Epoch 1/10
500/500 [==============================] - 237s 473ms/step - loss: 0.6710 - accuracy: 0.5842 - val_loss: 0.6563 - val_accuracy: 0.6018
Epoch 2/10
500/500 [==============================] - 269s 539ms/step - loss: 0.6593 - accuracy: 0.6022 - val_loss: 0.6444 - val_accuracy: 0.6235
Epoch 3/10
500/500 [==============================] - 269s 538ms/step - loss: 0.6561 - accuracy: 0.6069 - val_loss: 0.6416 - val_accuracy: 0.6220
Epoch 4/10
500/500 [==============================] - 269s 538ms/step - loss: 0.6519 - accuracy: 0.6146 - val_loss: 0.6481 - val_accuracy: 0.6093
Epoch 5/10
500/500 [==============================] - 234s 468ms/step - loss: 0.6526 - accuracy: 0.6134 - val_loss: 0.6390 - val_accuracy: 0.6215
Epoch 6/10
500/500 [==============================] - 235s 470ms/step - loss: 0.6486 - accuracy: 0.6141 - val_loss: 0.6465 - val_accuracy: 0.6118
Epoch 7/10
500/500 [==============================] - 270s 541ms/step - loss: 0.6457 - accuracy: 0.6193 - val_loss: 0.6368 -